# Generate input for positive class

In [2]:
import os
from tqdm import tqdm
from lxml import etree

In [3]:
path = '/Users/lmoncla/Documents/Data/Corpus/Choucas/Perdido'

In [57]:
words = []
for doc in tqdm(sorted(os.listdir(path))):
    filename = os.path.join(path, doc, doc+'.xml') 
    if os.path.exists(filename):
        tree = etree.parse(filename)


        for term in tree.xpath('.//rs[@type="place" and @subtype="ene"]/term[@type="place"]'):
            phrase = ''
            tokens = term.xpath('.//w[@pos="N" or @pos="PREPDET" or @pos="PREP"]')
            for i, w in enumerate(tokens):
                if len(w.text) > 1:
                    if (w.get('pos') != 'PREPDET') or ( (w.get('pos') == 'PREPDET' or w.get('pos') == 'PREP') and i < len(tokens)-1):
                        phrase += w.text.lower() + ' '
                
               
                #print(w.text, end=' ')
                if w.text == 'sculptures':
                    print(filename)
            words.append(phrase.strip())
            #print()

        # list to dict with frequency
        

100%|██████████| 96/96 [00:00<00:00, 261.80it/s]

/Users/lmoncla/Documents/Data/Corpus/Choucas/Perdido/FichierNettoye_75/FichierNettoye_75.xml


In [46]:
frequency_dict_geo = {}

for value in words:
    if value in frequency_dict_geo:
        frequency_dict_geo[value] += 1
    else:
        frequency_dict_geo[value] = 1

In [47]:
len(frequency_dict_geo)

93

In [48]:
frequency_dict_geo

{'chemin': 27,
 'lieu-dit': 16,
 'chemins': 1,
 'plaine': 46,
 'sommet de ligne': 1,
 'sommet': 30,
 'sentier': 39,
 'champ': 1,
 'col': 11,
 'rocher': 14,
 'sentiers': 1,
 'route': 52,
 'piste': 1,
 'lac': 47,
 'parking': 58,
 'intersection': 2,
 'auberge': 38,
 'port de chaussures': 1,
 'port': 1,
 'massif': 4,
 'vallée': 26,
 'crête': 4,
 'sommets': 3,
 'grotte': 13,
 'gare': 8,
 'place': 3,
 'ruisseau': 4,
 'parking du lac': 3,
 'rochers': 5,
 'source du lac': 1,
 'source': 31,
 'parking de route': 1,
 'ferme': 1,
 'refuge': 3,
 'chalet': 10,
 'auberge du lac': 1,
 'ruines': 32,
 'parking du château': 3,
 'château': 58,
 'barrage': 4,
 'villages': 3,
 'cascades': 1,
 'châteaux': 21,
 'abbaye': 24,
 'rue': 97,
 'rivière': 9,
 'carrefour': 54,
 'village': 14,
 'versant': 1,
 'sources': 1,
 'moulin': 3,
 'canal': 6,
 'forêt': 13,
 'auberges': 1,
 'chapelle': 20,
 'vallon': 3,
 'bourg': 2,
 'église': 9,
 'couvent': 9,
 'étang': 8,
 'étangs': 4,
 'cimetière': 8,
 'fontaine': 3,
 'cascad

In [23]:
for k, v in frequency_dict_geo.items():
    print(k, v)

chemin 27
lieu-dit 16
chemins 1
plaine 46
sommet de ligne 1
sommet 30
sentier 39
champ 1
col 11
rocher 14
sentiers 1
route 52
piste 1
lac 47
parking 58
intersection 2
auberge 37
port de chaussures 1
port 1
massif 4
vallée 26
crête 4
sommets 3
grotte 13
gare 8
place 3
ruisseau 4
parking du lac 3
rochers 5
source du lac 1
source 31
parking de route 1
ferme 1
refuge 3
chalet 10
auberge du lac 1
ruines 32
parking du château 3
château 57
barrage 4
villages 3
cascades 1
châteaux 21
abbaye 23
rue 97
rivière 9
carrefour 54
village 14
versant 1
sources 1
moulin 3
canal 6
forêt 13
auberges 1
chapelle 20
vallon 3
château du 1
bourg 2
église 9
couvent 9
étang 8
étangs 4
cimetière 8
fontaine 3
cascade 8
porte 14
commune 3
canton 1
département 5
territoire 1
pont 3
fossé 2
ruine 9
mur 2
parc 3
mont 3
région 5
ville 4
camping 6
carrefour source lac 1
hôtel 6
rives 1
parking route de 2
parking route 3
parking du col du 1
 1
sentier des sculptures situ de 1
montagne 1
l auberge 1
cour 3
forêts 2
l abba

In [25]:
words = []
for doc in tqdm(sorted(os.listdir(path))):
    filename = os.path.join(path, doc, doc+'.xml') 
    if os.path.exists(filename):
        tree = etree.parse(filename)

        for term in tree.xpath('.//rs[@type="unknown" and @subtype="ene"]/term[@type="unknown"]'):
            phrase = ''
            for w in term.xpath('.//w[@pos="N" or @pos="PREPDET" or @pos="PREP"]'):
                phrase += w.text.lower() + ' '
                #print(w.text, end=' ')
            words.append(phrase.strip())
            #print()

        # list to dict with frequency

100%|██████████| 96/96 [00:00<00:00, 206.73it/s]


In [31]:
frequency_dict_unknown = {}

for value in words:
    if value in frequency_dict_unknown:
        frequency_dict_unknown[value] += 1
    else:
        frequency_dict_unknown[value] = 1

In [32]:
frequency_dict_unknown

{'direction': 158,
 'hôtelier': 1,
 'panneau': 5,
 'information': 1,
 'nid': 1,
 'mitrailleuse': 1,
 'panneaux': 1,
 'ensemble': 1,
 'sud-ouest': 1,
 'poste': 3,
 'croix': 5,
 'cartes': 90,
 'nom': 4,
 'commandant': 1,
 'nécropole': 3,
 'balisage': 70,
 'première': 2,
 'seconde': 3,
 'statut': 2,
 'eaux': 3,
 'beauté': 1,
 'réserve': 2,
 'triangle sens descente direction': 1,
 'intersection balisage balisage triangle': 1,
 'triangle': 25,
 'paysages': 1,
 'pentes': 2,
 'descente': 1,
 'tour': 28,
 '|': 13,
 'ballon': 2,
 'bords': 1,
 'droite.longez': 1,
 'patte': 1,
 'oie': 1,
 'gazon': 6,
 'bifurcation': 4,
 'balisages': 28,
 'contreforts': 4,
 'visite': 9,
 'oeil': 5,
 'présence': 2,
 'col': 89,
 'salle': 5,
 'balisage disque': 8,
 'direction col': 2,
 'vignes': 2,
 'carte': 6,
 'panneau sentier': 2,
 'indication': 3,
 'sentier balisage anneau': 2,
 'logo': 2,
 'balisage chevalet': 4,
 'chevalet': 8,
 'direction rue': 2,
 'pied': 7,
 'en-dessus': 2,
 'second': 2,
 'vieux': 6,
 'membr

In [64]:
words = []
for doc in tqdm(sorted(os.listdir(path))):
    filename = os.path.join(path, doc, doc+'.xml') 
    if os.path.exists(filename):
        tree = etree.parse(filename)


        for term in tree.xpath('.//term//rs'):
            print(filename)
            phrase = ''
            for w in term.xpath('.//w'):
                phrase += w.text.lower() + ' '
        #print(phrase, end=' ')
        #print()
            
            #print()

        # list to dict with frequency
        

 29%|██▉       | 28/96 [00:00<00:00, 276.85it/s]

/Users/lmoncla/Documents/Data/Corpus/Choucas/Perdido/FichierNettoye_10/FichierNettoye_10.xml
/Users/lmoncla/Documents/Data/Corpus/Choucas/Perdido/FichierNettoye_10/FichierNettoye_10.xml
/Users/lmoncla/Documents/Data/Corpus/Choucas/Perdido/FichierNettoye_12/FichierNettoye_12.xml
/Users/lmoncla/Documents/Data/Corpus/Choucas/Perdido/FichierNettoye_12/FichierNettoye_12.xml
/Users/lmoncla/Documents/Data/Corpus/Choucas/Perdido/FichierNettoye_14/FichierNettoye_14.xml
/Users/lmoncla/Documents/Data/Corpus/Choucas/Perdido/FichierNettoye_14/FichierNettoye_14.xml
/Users/lmoncla/Documents/Data/Corpus/Choucas/Perdido/FichierNettoye_14/FichierNettoye_14.xml
/Users/lmoncla/Documents/Data/Corpus/Choucas/Perdido/FichierNettoye_14/FichierNettoye_14.xml
/Users/lmoncla/Documents/Data/Corpus/Choucas/Perdido/FichierNettoye_14/FichierNettoye_14.xml
/Users/lmoncla/Documents/Data/Corpus/Choucas/Perdido/FichierNettoye_14/FichierNettoye_14.xml
/Users/lmoncla/Documents/Data/Corpus/Choucas/Perdido/FichierNettoye_14

100%|██████████| 96/96 [00:00<00:00, 278.21it/s]

/Users/lmoncla/Documents/Data/Corpus/Choucas/Perdido/FichierNettoye_66/FichierNettoye_66.xml
/Users/lmoncla/Documents/Data/Corpus/Choucas/Perdido/FichierNettoye_66/FichierNettoye_66.xml
/Users/lmoncla/Documents/Data/Corpus/Choucas/Perdido/FichierNettoye_66/FichierNettoye_66.xml
/Users/lmoncla/Documents/Data/Corpus/Choucas/Perdido/FichierNettoye_66/FichierNettoye_66.xml
/Users/lmoncla/Documents/Data/Corpus/Choucas/Perdido/FichierNettoye_70/FichierNettoye_70.xml
/Users/lmoncla/Documents/Data/Corpus/Choucas/Perdido/FichierNettoye_70/FichierNettoye_70.xml
/Users/lmoncla/Documents/Data/Corpus/Choucas/Perdido/FichierNettoye_70/FichierNettoye_70.xml
/Users/lmoncla/Documents/Data/Corpus/Choucas/Perdido/FichierNettoye_70/FichierNettoye_70.xml
/Users/lmoncla/Documents/Data/Corpus/Choucas/Perdido/FichierNettoye_70/FichierNettoye_70.xml
/Users/lmoncla/Documents/Data/Corpus/Choucas/Perdido/FichierNettoye_72/FichierNettoye_72.xml
/Users/lmoncla/Documents/Data/Corpus/Choucas/Perdido/FichierNettoye_75